In [1]:
### IMPORTS ###
from mmap import mmap, ACCESS_READ
from struct import unpack
from math import sqrt
from glob import glob 
from os import chdir, listdir, makedirs
from os.path import splitext, isdir, exists
import numpy as np
import time
import pandas as pd

In [9]:
# b'\x00\x01\xc4\xb9\xff\xcb\xfb\xe4\x00\x83\x0b'
def unpacking_v2_format_hig(row):
 
    list_values = []
    index = (unpack('>I', bytes(row[0:4]))[0]) 
    acc_x = (unpack('>h', bytes(row[4:6]))[0]) / 1024
    acc_y = (unpack('>h', bytes(row[6:8]))[0]) / 1024
    acc_z = (unpack('>h', bytes(row[8:10]))[0]) / 1024

    gyr_x = (unpack('>h', bytes(row[10:12]))[0]) / 1024
    gyr_y = (unpack('>h', bytes(row[12:14]))[0]) / 1024
    gyr_z = (unpack('>h', bytes(row[14:16]))[0]) / 1024

    #index = (index / 6000)
    #list_values.append(index)
    acc_x = float(acc_x) 
    list_values.append(acc_x)
    acc_y = acc_y 
    list_values.append(acc_y)
    acc_z = acc_z 
    list_values.append(acc_z)
    amag = round(sqrt(pow(acc_x, 2) + pow(acc_y, 2) + pow(acc_z, 2)),2)
    list_values.append(amag)

    list_values.append(gyr_x)
    list_values.append(gyr_y)
    list_values.append(gyr_z)
    
    return list_values

def read_row(mm,hig=1):
    count = 0
    while True:
        count += 1
        if(hig==1):
            row = mm.read(17) # packet length
            if not len(row) == 17: 
                break 
        yield row 

def get_results_v2_format(hig_filename):
    complete_results = []    
    if hig_filename:
        print("getting results for", hig_filename)
        with open(hig_filename,'rb') as file:
            mm = mmap(file.fileno(), 0, access=ACCESS_READ)
            results = [unpacking_v2_format_hig(row) for row in read_row(mm)]
            complete_results.append(results)
            mm.close()
            file.close() 
        
    print("completed conversion",flush=True)
    return complete_results

In [23]:
#hig_filename = "data/mc_5_reps_test_1.HIG"
hig_filename = "../data/TKA_data/un_op/OST-0010ser13185648.OST"
#hig_filename = "data/mc_test_3_ml_60bpm.HIG"
#hig_filename = "data/lc_vag_test_1_60bpm.HIG"

converted_data = get_results_v2_format(hig_filename)
print(converted_data[0][0:10])

getting results for ../data/TKA_data/un_op/OST-0010ser13185648.OST
completed conversion
[[-0.06640625, -0.9609375, 0.2861328125, 1.0, 0.2451171875, 0.0625, -0.2314453125], [-0.0703125, -0.9501953125, 0.2783203125, 0.99, 0.2138671875, 0.0, -0.4072265625], [-0.056640625, -0.9716796875, 0.2734375, 1.01, 0.1513671875, 0.0244140625, -0.4189453125], [-0.0712890625, -0.9755859375, 0.2705078125, 1.01, 0.2080078125, -0.0947265625, -0.4619140625], [-0.0810546875, -0.9638671875, 0.291015625, 1.01, 0.2578125, -0.076171875, -0.1884765625], [-0.0830078125, -0.9658203125, 0.2705078125, 1.01, 0.3525390625, 0.005859375, -0.4130859375], [-0.076171875, -0.970703125, 0.2802734375, 1.01, 0.1884765625, 0.0498046875, -0.4248046875], [-0.0791015625, -0.9794921875, 0.275390625, 1.02, 0.232421875, 0.0185546875, -0.498046875], [-0.06640625, -0.9697265625, 0.263671875, 1.01, 0.2138671875, -0.0634765625, -0.3037109375], [-0.0439453125, -0.9716796875, 0.263671875, 1.01, -0.0625, -0.0888671875, 0.3515625]]


In [24]:
# write to csv
columns = ['x', 'y', 'z', 'mag', 'gyr_x', 'gyr_y', 'gyr_z']

# Create the DataFrame
df = pd.DataFrame(converted_data[0], columns=columns)
print(df.head(5))

df.to_csv("../data/TKA_data/un_op/tka_un_op_3.csv", index=False)

          x         y         z   mag     gyr_x     gyr_y     gyr_z
0 -0.066406 -0.960938  0.286133  1.00  0.245117  0.062500 -0.231445
1 -0.070312 -0.950195  0.278320  0.99  0.213867  0.000000 -0.407227
2 -0.056641 -0.971680  0.273438  1.01  0.151367  0.024414 -0.418945
3 -0.071289 -0.975586  0.270508  1.01  0.208008 -0.094727 -0.461914
4 -0.081055 -0.963867  0.291016  1.01  0.257812 -0.076172 -0.188477
